# Eat Safe, Love

## Notebook Set Up

In [30]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [31]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [32]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [33]:
# review the collections in our database
collection_names = db.list_collection_names()
print("Collections in the database:")
for name in collection_names:
    print(name)

Collections in the database:
establishments


In [34]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [35]:
query = {"scores.Hygiene": 20}

# Use count_documents
num_documents = establishments.count_documents(query)
print("Number of documents with a hygiene score of 20:", num_documents)

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
print("First document with a hygiene score of 20:")
pprint(first_document)

Number of documents with a hygiene score of 20: 82
First document with a hygiene score of 20:
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65dab84d84255d92de9fa890'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/estab

In [36]:
df = pd.DataFrame(list(establishments.find(query)))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  65dab84d84255d92de9fa890   110681                  0   
1  65dab84d84255d92de9fac0e   612039                  0   
2  65dab84d84255d92de9faf1d   730933                  0   
3  65dab84d84255d92de9fb10d   172735                  0   
4  65dab84d84255d92de9fb119   172953                  0   
5  65dab84d84255d92de9fbab6   512854                  0   
6  65dab84d84255d92de9fbcd9  1537089                  0   
7  65dab84e84255d92de9fd203   155648                  0   
8  65dab84e84255d92de9fd64f  1012883                  0   
9  65dab84e84255d92de9fde59   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532       

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [39]:
# Construct the query to find establishments with London as the Local Authority and RatingValue >= 4
query = {
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
result_count = establishments.count_documents(query)
print("Number of documents:", result_count)

# Display the first document in the results using pprint
result_document = establishments.find_one(query)

if result_document:
    pprint(result_document)
else:
    print("No documents found matching the query criteria.")


Number of documents: 0
No documents found matching the query criteria.


In [43]:
# Convert the query result to a DataFrame
df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [48]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
establishments_collection = db['establishments']

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
results = establishments_collection.find(query).sort(sort).limit(limit)
for result in results:
    pprint(result)
    
# Fetch the results
results = establishments_collection.find(query).sort(sort).limit(limit)

# Convert the results to a list of dictionaries
result_list = list(results)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(result_list)

# Display the DataFrame
print(df)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [52]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results

# Construct the aggregation pipeline
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

# Execute the aggregation pipeline
result_cursor = establishments_collection.aggregate(pipeline)

# Convert the cursor to a list of results
results = list(result_cursor)

# Print the first ten results using pprint
for result in results:
    pprint(result)


{'_id': 'Thanet', 'count': 2260}
{'_id': 'Greenwich', 'count': 1764}
{'_id': 'Maidstone', 'count': 1426}
{'_id': 'Newham', 'count': 1422}
{'_id': 'Swale', 'count': 1372}
{'_id': 'Chelmsford', 'count': 1360}
{'_id': 'Medway', 'count': 1344}
{'_id': 'Bexley', 'count': 1214}
{'_id': 'Southend-On-Sea', 'count': 1172}
{'_id': 'Tendring', 'count': 1084}


In [53]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 10
First 10 rows of the DataFrame:
               _id  count
0           Thanet   2260
1        Greenwich   1764
2        Maidstone   1426
3           Newham   1422
4            Swale   1372
5       Chelmsford   1360
6           Medway   1344
7           Bexley   1214
8  Southend-On-Sea   1172
9         Tendring   1084
